In [1]:
from pyspark import SparkContext
import numpy as np
import pandas as pd
import re

In [79]:
# Pendant les test pour stopper un SparkContext
#sc.stop()

In [2]:
# Connexion au cluster / test en local
sc = SparkContext("local", "Page Rank With PySpark")

In [3]:
# Préparation du fichier avant utilisation dans l'algo
# Taille du fichier initial = dfFile.shape = (5105039, 2)
#dfFile = pd.read_csv("web-Google.txt", sep='\t', header=3)
# Pour les tests récupérations des 100 premières lignes uniquement 
dfFile = pd.read_csv("web-Google.txt", sep='\t', header=3, nrows = 20)
#dfFile.rename(columns={'# FromNodeId':'FromNodeId'}, inplace = True)
print(dfFile.shape)
#dfFile.to_csv("web-GoogleSmall.txt", sep='\t', header=False, index=False)
dfFile.to_csv("web-GoogleSmall.txt", sep=' ', header=False, index=False)

(20, 2)


In [4]:
# Read the text file "GoogleReduc.txt" from a local file system (available on all nodes),
# and return it as an RDD of Strings.
RddDataBase = sc.textFile("web-GoogleSmall.txt")
#RddDataBase.collect()

In [5]:
def linkNodesFromTo(NodesRow):
    nodeslinked = re.split(' ', NodesRow)
    return nodeslinked[0], nodeslinked[1]

In [6]:
# First step :
# Create key/value pairs, 
# where the key is the name of the page and the value is out-links from the page (Di) 
# and İnitiate PageRank values (Ri) as 1/Number of pages.
#links = sc.textFile("web-GoogleSmall.txt")

# Key/value pairs
links = RddDataBase.map(lambda NodesRow: linkNodesFromTo(NodesRow)).distinct().groupByKey().cache()
#links = links.map(lambda x: (x.split(' ')[0], x.split(' ')[1:]))
print(links.collect())

# Find node count
N = links.count()
print(N)

# Create and initialize the ranks
ranks = links.map(lambda node: (node[0],1.0/N))
print(ranks.collect())

[('0', <pyspark.resultiterable.ResultIterable object at 0x0000016C9E69DA60>), ('11342', <pyspark.resultiterable.ResultIterable object at 0x0000016C9E69DB20>), ('824020', <pyspark.resultiterable.ResultIterable object at 0x0000016C9E69DB80>)]
3
[('0', 0.3333333333333333), ('11342', 0.3333333333333333), ('824020', 0.3333333333333333)]


In [110]:
#Map: For each node i, 
#calculate vote (Ri/Di) for each out-link of i and propagate to adjacent nodes.
#Reduce: For each node i, sum the upcoming votes and update Rank value (Ri).
#Repeat this Map-Reduce step until Rank values converge (stable or within a margin).
nbIter=5
for i in range(nbIter):
    # Join graph info with rank info and propogate to all neighbors rank scores (rank/(number of neighbors)
    # And add up ranks from all in-coming edges
    ranks = links.join(ranks).flatMap(lambda x : [(i, float(x[1][1])/len(x[1][0])) for i in x[1][0]])\
    .reduceByKey(lambda x,y: x+y)
    print(ranks.sortByKey().collect())

[('0', 0.19047619047619047), ('11342', 0.08333333333333333), ('27469', 0.023809523809523808), ('309564', 0.023809523809523808), ('322178', 0.023809523809523808), ('38716', 0.023809523809523808), ('387543', 0.023809523809523808), ('427436', 0.023809523809523808), ('538214', 0.023809523809523808), ('638706', 0.023809523809523808), ('645018', 0.023809523809523808), ('824020', 0.08333333333333333), ('835220', 0.023809523809523808), ('856657', 0.023809523809523808), ('867923', 0.10714285714285714), ('891835', 0.10714285714285714), ('91807', 0.16666666666666666)]
[('0', 0.047619047619047616), ('11342', 0.047619047619047616), ('27469', 0.005952380952380952), ('309564', 0.005952380952380952), ('322178', 0.005952380952380952), ('38716', 0.005952380952380952), ('387543', 0.005952380952380952), ('427436', 0.005952380952380952), ('538214', 0.005952380952380952), ('638706', 0.005952380952380952), ('645018', 0.005952380952380952), ('824020', 0.047619047619047616), ('835220', 0.005952380952380952), (